# Møller–Plesset partitioning

Using Rayleigh–Schrödinger perturbation theory ([RSPT](pt)), the Hamiltonian can be partitioned according to the Møller–Plesset (MP) scheme {cite}`Szabo2012,Helgaker2014,mbpt2009`

\begin{equation*}
\hat{H} = \hat{F} + \hat{\Phi}
\end{equation*}

where the zeroth-order Hamiltonian, $\hat{F}$, is the Fock operator and the perturbation, $\hat{\Phi}$, is known as the fluctuation potential.

The exact solutions to the zeroth-order Hamiltonian are found from the time-independent Schrödinger equation

\begin{align*}
\hat{F} | \Phi_n \rangle  &= {\cal E}_n | \Phi_n \rangle 
\end{align*}

The exact state solutions to this equation are given by the Hartree–Fock state and its excited determinants

\begin{align*}
| \Phi_0 \rangle & = | \Psi_\mathrm{HF} \rangle \\
| \Phi_n \rangle & = | \Psi_{ij\ldots}^{ab\ldots} \rangle; \quad n > 0
\end{align*}

and, in terms of orbital energies, the associated exact energies are equal to

\begin{align*}
{\cal E}_0 & = \sum_{i}^\mathrm{occ} \varepsilon_{i} \\ 
{\cal E}^{ab\cdots}_{ij\cdots} &= \varepsilon_{a} + \varepsilon_{b} + \ldots - \varepsilon_{i} - \varepsilon_{j} - \ldots \\
\end{align*}

## Perturbation expansion in the fluctuation potential

The exact ground state is a solution to

$$
\hat{H} | \Psi \rangle = E | \Psi \rangle
$$

where $| \Psi \rangle$ and $E$ are expanded in orders of the perturbation

$$
| \Psi \rangle = \sum_{k=0}^\infty | \Psi^{(k)} \rangle; \qquad E = \sum_{k=0}^\infty E^{(k)}
$$

From RSPT, we get 

\begin{align*}
E^{(0)} & = {\cal E}_0 \\
E^{(1)} & = \langle \Psi_\mathrm{HF} | \hat{\Phi} | \Psi_\mathrm{HF}  \rangle = -\frac{1}{2} \sum_{i,j}^\mathrm{occ} \langle ij \| ij \rangle
\end{align*}

so that 

$$
E_{\mathrm{HF}} = E^{(0)} + E^{(1)}
$$

The first-order correction to the wave function is obtained from the [RSPT master equation](pt)

$$
| \Psi^{(1)} \rangle = -\frac{1}{4} \sum_{i,j}^\mathrm{occ} \sum_{a,b}^\mathrm{virt} t_{ij}^{ab} | \Psi_{ij}^{ab} \rangle ; \qquad
t_{ij}^{ab} = \frac{\langle ab \| ij \rangle}{{\cal E}_{ij}^{ab}}
$$

and, in terms of the introduced $t$-amplitudes, we thereby get a second-order energy correction equal to

$$
E^{(2)}  = - \frac{1}{4} \sum_{i,j}^\mathrm{occ} \sum_{a,b}^\mathrm{virt} t_{ij}^{ab} \langle ij || ab \rangle
$$

such that

$$
E_{\mathrm{MP2}} = E_{\mathrm{HF}} + E^{(2)}
$$

### Closed-shell reference state

For a closed-shell Hartree–Fock state, the MP2 energy correction can be written in terms of sums referring to molecular orbitals (MOs)

$$
E^{(2)} = E_{\mathrm{os}}^{(2)} + E_{\mathrm{ss}}^{(2)}
$$

where 

\begin{align*}
E_{\mathrm{os}}^{(2)} & = -
\sum_{i,j}^\mathrm{occ} \sum_{a,b}^\mathrm{virt}
\frac{(ia|jb)(ia|jb)}{{\cal E}_{ij}^{ab}} \\
E_{\mathrm{ss}}^{(2)} & = -
\sum_{i,j}^\mathrm{occ} \sum_{a,b}^\mathrm{virt}
\frac{(ia|jb)}{{\cal E}_{ij}^{ab}}
\big[ (ia|jb) - (ib|ja) \big] \\
\end{align*}

The [chemist's notation](https://kthpanor.github.io/echem/docs/units_notation.html#two-electron-integrals) has here been used for the integrals.

# Illustration

Let us illustrate the theory with a calculation of the MP2 energy of water.

In [ ]:
import veloxchem as vlx

In [ ]:
h2o_xyz = """3
water                                                                                                                          
O    0.000000000000        0.000000000000        0.000000000000                         
H    0.000000000000        0.740848095288        0.582094932012                         
H    0.000000000000       -0.740848095288        0.582094932012
"""

molecule = vlx.Molecule.read_xyz_string(h2o_xyz)
basis = vlx.MolecularBasis.read(molecule, "cc-pvdz", ostream=None)

nocc = molecule.number_of_alpha_electrons()
nvirt = basis.get_dimension_of_basis() - nocc

print("Number of occupied orbitals:", nocc)
print("Number of unoccupied orbitals:", nvirt)

In [ ]:
molecule.show()

## Hartree–Fock reference state

In [ ]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()
scf_results = scf_drv.compute(molecule, basis)

We obtain the orbital energies.

In [ ]:
E = scf_results["E_alpha"]

## Integrals in the MO basis

We create an instance of the `MOIntegralsDriver` class to get the electron repulsion integrals in the MO basis and in the chemist's notation.

In [ ]:
erimo_drv = vlx.MOIntegralsDriver()

ovov = erimo_drv.compute_in_memory(molecule, basis, scf_drv.mol_orbs, "chem_ovov")
print("(ov|ov):", ovov.shape)

## The MP2 energy correction

We now have all the ingredients to compute the opposite-spin and same-spin components of the MP2 energy correction.

In [ ]:
e_mp2_ss = 0.0
e_mp2_os = 0.0

# extract the occupied subset of the orbital energies
e_o = E[:nocc]
# extract the virtual subset of the orbital energies
e_v = E[nocc:]

for i in range(nocc):
    for j in range(nocc):
        for a in range(nvirt):
            for b in range(nvirt):
                # enegy denominators
                e_ijab = e_v[a] + e_v[b] - e_o[i] - e_o[j]

                # update opposite-spin component of the energy
                e_mp2_os -= (ovov[i, a, j, b] * ovov[i, a, j, b]) / e_ijab

                # update same-spin component of the energy
                e_mp2_ss -= (
                    ovov[i, a, j, b] * (ovov[i, a, j, b] - ovov[i, b, j, a]) / e_ijab
                )

In [ ]:
print(f"Opposite-spin MP2 energy: {e_mp2_os:12.8f}")
print(f"Same-spin MP2 energy    : {e_mp2_ss:12.8f}")
print(f"MP2 energy              : {e_mp2_os + e_mp2_ss:12.8f}")

Let us compare this result with the MP2 energy obtained with VeloxChem.

In [ ]:
mp2_drv = vlx.Mp2Driver()
mp2_drv.ostream.mute()

mp2_results = mp2_drv.compute_conventional(molecule, basis, scf_drv.mol_orbs)

In [ ]:
print(f"Energy difference: {mp2_results['mp2_energy']:12.8f}")

We note that the two results are in perfect agreement.

(sec:mp2-size-consistency)=
# Size consistency

We see that to second order, Møller-Plesset perturbation theory only involves up to double excitations from the HF reference. It would thus be natural to consider it an approximation to CISD, and expect it to suffer from the same issue, namely a lack of size consistency. Yet this is not the case as discussed in the [RSPT section on this topic](https://kthpanor.github.io/echem/docs/elec_struct/pt.html#size-extensivity-in-rspt).

In [ ]:
dimer_xyz = """6
2 water 100 Å apart                                                                                                            
O    0.000000000000        0.000000000000        0.000000000000                         
H    0.000000000000        0.740848095288        0.582094932012                         
H    0.000000000000       -0.740848095288        0.582094932012
O  100.000000000000        0.000000000000        0.000000000000                         
H  100.000000000000        0.740848095288        0.582094932012                         
H  100.000000000000       -0.740848095288        0.582094932012
"""

molecule = vlx.Molecule.read_xyz_string(dimer_xyz)
basis = vlx.MolecularBasis.read(molecule, "cc-pvdz", ostream=None)

In [ ]:
molecule.show()

In [ ]:
scf_results = scf_drv.compute(molecule, basis)

mp2_dimer_results = mp2_drv.compute_conventional(molecule, basis, scf_drv.mol_orbs)

In [ ]:
print("=" * 30)
print("MP2 energies")
print("-" * 30)
print(f"Dimer       : {mp2_dimer_results['mp2_energy']:16.10f}")
print(f"Two monomers: {2 * mp2_results['mp2_energy']:16.10f}")
print("=" * 30)

We can see that the two energies match. MP2 is size consistent! So why is it behaving better than CISD in this aspect?

The key is that in MP2, the coefficients of the excited determinants are independent of the system size. Thus a molecule would have the same MP2 energy correction regardless of the presence or not of another, non-interacting, molecule. By contrast, in CISD, the coefficients depend on the system size through normalization, lowering the weight of these determinants as the size of the system increases.